In [ ]:
import os
import yaml
from data_fetcher import YahooFinanceDataFetcher
from mt5_fetcher import MT5DataFetcher
from google_sheet_api import GoogleSheetsUploader
from analyzer import run_analysis, run_all_analyses

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# === Load Configuration === #
def load_config(path):
    with open(path, "r") as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# === Extract Parameters with Defaults === #
fetch_mt5 = config.get("fetch_mt5", True)
params = {
    "std_multiplier": config.get("std_multiplier", 1.96),
    "initial_balance": config.get("initial_balance", 1000),
    "invest_per_week": config.get("invest_per_week", 100),
    "leverage": config.get("leverage", 1000),
    "min_years_required": config.get("min_years_required", 1)
}

def main():
    print("📥 Fetching data from Yahoo Finance...")
    fetcher = YahooFinanceDataFetcher(CONFIG_PATH)
    full_df = fetcher.get_data()
    symbol_list = full_df["Symbol"].unique()

    # Run analysis for one example symbol
    symbol = "GOOGL.NAS"
    df, dd_thresh, gain_thresh, daily_chg = run_analysis(
        df=full_df,
        symbol=symbol,
        plots_dir=PLOTS_DIR,
        coeff=fetcher.coeff_map.get(symbol),
        **params
    )
    
    # Run all symbol analyses
    df_summary, df_final = run_all_analyses(
        full_df=full_df,
        symbol_list=symbol_list,
        plots_dir=PLOTS_DIR,
        coeff_map=fetcher.coeff_map,
        **params
    )

    # Optionally fetch live prices from MT5
    if fetch_mt5:
        print("🔄 Fetching MT5 live prices...")
        df = MT5DataFetcher(df_summary, params["min_years_required"])

    # Upload results to Google Sheets
    try:
        print("📤 Uploading to Google Sheets...")
        uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
        uploader.upload_dataframe(df, "Overview", replace=False)
        print("✅ Upload successful!")
    except Exception as e:
        print(f"❌ Upload failed: {e}")

if __name__ == "__main__":
    main()

📥 Fetching data from Yahoo Finance...
📈 Fetching AAPL.NAS (AAPL)...
📈 Fetching AAVEUSD (AAVE-USD)...
📈 Fetching ABBV.NYSE (ABBV)...
📈 Fetching ABNB.NAS (ABNB)...
📈 Fetching ABT.NYSE (ABT)...
📈 Fetching ACN.NYSE (ACN)...
📈 Fetching ADAUSD (ADA-USD)...
📈 Fetching ADBE.NAS (ADBE)...
📈 Fetching ADS.XE (ADS.DE)...
📈 Fetching AI.EPA (AI.PA)...
📈 Fetching AIR.EPA (AIR.PA)...
📈 Fetching ALGOUSD (ALGO-USD)...
📈 Fetching ALV.XE (ALV.DE)...
📈 Fetching AMD.NAS (AMD)...
📈 Fetching AMZN.NAS (AMZN)...
📈 Fetching APTUSD (APT21794-USD)...
📈 Fetching ASML.EAS (ASML.AS)...
📈 Fetching ATOMUSD (ATOM-USD)...
📈 Fetching AUDCAD (AUDCAD=X)...
📈 Fetching AUDCHF (AUDCHF=X)...
📈 Fetching AUDJPY (AUDJPY=X)...
📈 Fetching AUDNZD (AUDNZD=X)...
📈 Fetching AUDUSD (AUDUSD=X)...
📈 Fetching AUS200 (^AXJO)...
📈 Fetching AVAXUSD (AVAX-USD)...
📈 Fetching AVGO.NAS (AVGO)...
📈 Fetching AXP.NYSE (AXP)...
📈 Fetching AXSUSD (AXS-USD)...
📈 Fetching AZN.LSE (AZN.L)...
📈 Fetching BA.NYSE (BA)...
📈 Fetching BABA.NYSE (BABA)...
📈 Fetc